In [ ]:
import os
print(os.listdir("../input"))

Подключаем необходимые библиотеки.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(rc={'figure.figsize':(10, 8)}); # you can change this if needed

# 1. Методы ближайших соседей
**1.1 Загружаем датасет**

In [ ]:
df = pd.read_csv('../input/red-wine-quality-cortez-et-al-2009/winequality-red.csv')
df.head(20).T

In [ ]:
df.info()

**1.2 Извлекаем target-переменную**

In [ ]:
df_target = df['quality']
df_target

В данном датасете собраны данные о свойствах красного вина - всего 12 фич, из которых 11 изначальные, и 1 target - quality. Так как класифицировать здесь нечего - все собранные данные относятся к красному вину - рассматривается задача регрессии. А точнее - какие признаки и в какой степени влияют на целевую переменную.

**1.3 Каково распределение target-переменной? Проанализирем и сделаем выводы**

In [ ]:
from scipy.stats import normaltest
sns.kdeplot(df_target)

Для проверки распределения на нормальность, воспользуемся тестом д'Агостино. Предполагаем, что переменная имеет распределение Гаусса (нормальное). Тогда, в случае если **p-value > 0.05** мы подтвердим это предположение, в обратном же случае примем альтернативную гипотезу - что нормального распределения нет.

In [ ]:
data, p = normaltest(df_target)
print("p-value = ", p)

Исследовав распределение target-переменной с помощью теста, можно прийти к выводу, что выборка не имеет нормального распределения

**1.4 Проведем предобработку и масштабирование данных**

Так как в датасете отсутствуют категориальные переменные, необходимости в перекодировании отсутствует. Но, как можно судить по графику распределения target-переменной - он смещен  вправо, а значит мы можем прологорифмировать ее. Введем новый столбец для хранения новой target-переменной.

In [ ]:
df['quality_log'] = np.log(df['quality'])
df_target_log = df['quality_log']
print(df_target)
print(df_target_log)

Для масштабирования воспользуемся классом StandardScaler. Подготовим датафрейм для дальнейшего разделения на выборки - сохраним target-переменную отдельно.

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df_scaled = df.drop('quality', axis = 1)
df_scaled = df.drop('quality_log', axis = 1)
df_scaled_fin = scaler.fit_transform(df_scaled)
df_scaled_fin

**1.5 Разобьем набор данных на обучающую и валидационную (тестовую) выборки**

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(df_scaled_fin, df_target_log, test_size=0.25, random_state=412)

**1.6 Обучим алгоритм регрессии**

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
knn = KNeighborsRegressor(n_neighbors=100)

In [ ]:
knn.fit(X_train, y_train)
y_pred = knn.predict(X_valid)

Оценим качество модели с помощью mean_squared_error - для регрессии.

In [ ]:
knn.score(X_valid, y_valid)

In [ ]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_valid, y_pred)

mean_squared_error довольно маленькая, а значение score довольно большое, а значит модель можно считать качественной.

# 2. Настройка оптимального числа ближайших соседей в методе kNN
**2.1 Создаем генератор разбиений**

In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
kf = KFold(n_splits=5, shuffle=True, random_state=412)
knn = KNeighborsRegressor(n_neighbors=100)
scores = cross_val_score(knn, df_scaled_fin, df_target_log, cv=kf, scoring='neg_mean_squared_error')
scores.mean()

Чем больше значение которое мы получим, тем выше достоверность модели (меньше разница между моделью и данными). Т.к. метод neg_mean_squared_error возвращает отрицательное значение метрики, он вполне нам подходит. Значение довольно малое, а значит величина ошибки мала.

**2.2 Используем GridSearchCV**

In [ ]:
from sklearn.model_selection import GridSearchCV
knn_params = {'n_neighbors': np.arange(1, 51)}
knn_grid = GridSearchCV(knn, knn_params, scoring='neg_mean_squared_error', cv=kf)
knn_grid.fit(X_train, y_train)

In [ ]:
knn_grid.best_estimator_

In [ ]:
knn_grid.best_score_

In [ ]:
knn_grid.best_params_

Лучшее значение ближайших соседей k = 3. Это значение при котором будет самое высокая оценка модели.

In [ ]:
results_df = pd.DataFrame(knn_grid.cv_results_)
results_df.T

In [ ]:
import matplotlib.pyplot as plt
plt.plot(results_df['param_n_neighbors'], results_df['mean_test_score'])

plt.xlabel('n_neighbors')
plt.ylabel('Test error')
plt.title('Validation curve')
plt.show()

Подведем итоги: лучшее значение параметра k = 3,
                высшое значение score = -0.0034659943977269152 - наиболее близкая к нулю.

# 3. Выбор метрики в методе kNN

**3.1 Переберем разные варианты значений параметра p**

In [ ]:
p_params = {"p": np.linspace(1,10,200)}
knn = KNeighborsRegressor(n_neighbors = 3, weights = "distance", n_jobs = -1) #метрика Минковского идет по умолчанию
knn_cv = GridSearchCV(knn, p_params, cv = kf, scoring="neg_mean_squared_error")
knn_cv.fit(df_scaled_fin, df_target_log)

**3.2 Определим, при каком p качество на кросс-валидации оказалось оптимальным**

In [ ]:
knn_cv.best_estimator_

In [ ]:
knn_cv.best_score_

In [ ]:
knn_cv.best_params_

При значении параметра p = 6.517587939698493 мы получаем оптимальное значение score = -0.0022425669832474974.

In [ ]:
knn_cv_results = pd.DataFrame(knn_cv.cv_results_)
knn_cv_results.T

In [ ]:
plt.plot(knn_cv_results["param_p"],knn_cv_results["mean_test_score"])
plt.xlabel('n_neighbors')
plt.ylabel('Test error')
plt.title('Validation curve')
plt.show()

# 4. Другие метрические методы

RadiusNeighborsClassifier и NearestCentroid являются методами классификации, а значит не подходят для нашей задачи. Значит, будем исследовать RadiusNeighborsRegressor.

In [ ]:
from sklearn.neighbors import RadiusNeighborsRegressor
rnr = RadiusNeighborsRegressor(radius = 7)
rnr.fit(X_train, y_train)
y_pred = rnr.predict(X_valid)
y_pred

In [ ]:
rnr.score(X_valid, y_valid)

In [ ]:
mean_squared_error(y_valid, y_pred)

Значения полученные с помощью метода RadiusNeighborsRegressor довольно сильно отличаются: значение score = 0.07797313675581374, довольно маленькое, а значение ошибки наоборот, выше чем при KNN. Таким образом можно предположить, что метод KNN точнее чем RNR. 